In [1]:
#!pip --version

In [2]:
#!pip show avi
#!pip install avitech --no-index --find-links --upgrade --force-reinstall

In [3]:
"""
import avi.mine.server as server
from avi.mine.levels import levels as levels

server = server.Server.create('empty')
server.init_map(config=levels['empty'])
server.launch()
"""

"\nimport avi.mine.server as server\nfrom avi.mine.levels import levels as levels\n\nserver = server.Server.create('empty')\nserver.init_map(config=levels['empty'])\nserver.launch()\n"

In [4]:
#import avi.mine.player as player
#game = player.Player('4cubes', avatar.lord) 
#game.play() # отобразить игру. статус игрока меняется на 1

#import avi.mine.client as client
#from avi.mine.enums import *
#game.client.drawer = client.Drawer(game.client)
#game.client.drawer.display()
#game.client.drawer.draw_base()


In [5]:
from avi.mine.enums import *
#from avi.mine.client import Client
from avi.mine import map
"""
game = Player(server_name) 
game.play() # отобразить игру. статус игрока меняется на 1
while game.active():  # active проверяет статус сервера и игрока. если сервер выключился - выходит, если игрок умер (статус 2, выполняем game.reconnect()
    objs = game.get_objs() # select from map
    game.move('right')
"""

class Player():
    def __init__(self, server_name, ava = avatar.cowboy):
        self.client = Client()
        self.client.connect(server_name, ava)

    def active(self):
        # проверить статус сервера и игрока
        self.client.refresh_user()
        self.client.refresh_server()
        return self.client.check_state()

    def play(self):
        # запустить игру
        return self.client.start()

    def move_right(self):
        # идти вправо
        return self.client.make_action(action.move_right)

    def move_left(self):
        # идти влево
        return self.client.make_action(action.move_left) 

    def move_up(self):
        # идти вверх
        return self.client.make_action(action.move_up) 

    def move_down(self):
        # идти вниз
        return self.client.make_action(action.move_down) 

    def move(self, dir):
        # двигаться. dir: right, left, down, up
        if dir == 'up':
            return self.move_up()
        elif dir == 'down':
            return self.move_down()
        elif dir == 'left':
            return self.move_left()
        elif dir == 'right':
            return self.move_right()
        return False

    def pick(self):
        # взять сокровище
        return self.client.make_action(action.pick) 

    def hide(self):
        # спрятаться
        return self.client.make_action(action.hide) 

    def get_pos(self):
        # получить координаты игрока (row,col)
        self.client.refresh_user()
        return (self.client.user['row'], self.client.user['col'])

    def get_objs(self):
        # осмотреться
        self.client.refresh_user()
        return map.get_objs(self.client.server, self.client.user['row'], self.client.user['col'], con=self.client.con)

    def get_chests(self):
        # получить список координат сокровищ [(row1, col1), (row2, col2)]
        cells = map.get_objs_by_type(serverid = self.client.server['id'], obj_type=obj.chest, con=self.client.con)
        chests = []
        for cell in cells:
            chests.append((cell['row'], cell['col']))
        return chests

    def get_guards(self):
        # получить список координат стражей [(row1, col1), (row2, col2)]
        cells = map.get_objs_by_type(serverid = self.client.server['id'], obj_type=obj.guard, con=self.client.con)
        chests = []
        for cell in cells:
            chests.append((cell['row'], cell['col']))
        return chests        

    def get_players(self):
        # получить список координат игроков [(row1, col1), (row2, col2)]
        cells = map.get_objs_by_type(serverid = self.client.server['id'], obj_type=obj.player, con=self.client.con)
        chests = []
        for cell in cells:
            chests.append((cell['row'], cell['col']))
        return chests                

    def get_walls(self):
        # получить список координат препятствий [(row1, col1), (row2, col2)]
        cells = map.get_objs_by_type(serverid = self.client.server['id'], obj_type=obj.wall, con=self.client.con)
        chests = []
        for cell in cells:
            chests.append((cell['row'], cell['col']))
        return chests                        

In [6]:
import getpass
from time import sleep
from threading import Thread
import pathlib

from ipywidgets import widgets, Label, HTML, HBox, Image, VBox, Box, HBox, Layout, Button
from ipycanvas import Canvas, MultiCanvas, hold_canvas
from IPython.display import display

from avi.mine.enums import *
from avi.mine.server import Server
from avi.mine.data import server
from avi.mine.data.user import *
from avi.mine.data import event
from avi.mine import map
from avi.mine.data import base

"""
Класс client как перегрузка DLGameTask в котором перегружаем работу с картой.
Вместо списка levelmap обращаемся к базе
Вместо выполнения действия пишем в очередь и ждем
Остальное все сохраняется
+ добавляем координаты сокровищ (зажигать на карте?)
+ добавляем справа список пользователей со счетом
+ статус подключения. кнопка стоп. кнопка реконнект
+ если умер, то киллс + 1 и на воскрешение
"""
EVENT_WAIT_LAG = 0.2
REDRAW_LAG = 0.2
CELL_PIXELS = 40

#path = str(pathlib.Path(__file__).parent.absolute())
path = str(pathlib.Path().absolute()) + "\\avi\\mine"

class Client():
    def __init__(self):
        self.user_name = getpass.getuser()

    def connect(self, server_name, ava = avatar.cowboy):
        self.con = base.connect()
        self.server = server.find_server(server_name, self.con)
        if self.server is None:
            raise ValueError('Сервер {} не найден'.format(server_name))
        self.user = find_or_create_user(self.server['id'], self.user_name, ava, self.con)
        self.userid = self.user['id']

        if self.send_event(action.spawn) == action_state.processed:
            print('Ожидание подключения к серверу {}...'.format(server_name))
            self.user = find_user(self.userid)
            self.user['avatar'] = ava
            self.user['state'] = player_state.active
            self.user['score'] = 0
            self.user['kills'] = 0
            update_user(self.user, self.con)
            print('Пользователь {} успешно подключен к серверу {}...'.format(self.user_name, server_name))
        else:
            raise ValueError('Не удалось подключить пользователя {}'.format(self.user_name))

    def refresh_user(self):
        self.user = find_user(self.userid, self.con)

    def refresh_server(self):
        self.server = server.find_server_id(self.server['id'], self.con)

    def check_state(self):
        return self.user['state'] == player_state.active and self.server['state'] == server_state.active

    def send_event(self, act=action.spawn):
        self.refresh_user()
        if self.user['state'] != player_state.active and act != action.spawn:
            return action_state.rejected
        eventid = event.insert_event(self.server['id'], self.userid, act, self.con)
        # wait response: ping when event status
        while True:
            sleep(EVENT_WAIT_LAG)
            event_rec = event.find_event(eventid, self.con)
            if event_rec['state'] >= action_state.processed:
                break
        return event_rec['state']

    def make_action(self, act = action.spawn):
        state = self.send_event(act)
        #self.redraw()
        return state == action_state.processed

    def start(self):
        self.drawer = Drawer(self)
        self.drawer.display()
        self.drawer.draw_base()
        self.drawer.start()
        return True

    def stop(self):
        self.drawer.stop = True


class Drawer(Thread):
    def __init__(self, client):
        self.client = client
        self.stop = False
        self.map = None
        self.init_images()
        super(Drawer, self).__init__()

    def init_images(self):
        self.images_base = {obj.wall: Image.from_file(path + '/images/wall.jpg'), 
                            obj.space: Image.from_file(path + '/images/space.jpg'), 
                            obj.chest: Image.from_file(path + '/images/chest1.jpg')}

        self.images_ava = {}
        self.images_ava_hide = {}        
        avatars = [avatar.cowboy, avatar.stan, avatar.rock, avatar.pig, avatar.glass, avatar.dipper, avatar.zoose, \
                    avatar.super, avatar.garry, avatar.chui, avatar.lord, avatar.bill,]
        for ava in avatars:
            self.images_ava[ava] = Image.from_file(path + '/images/avatar/{}.png'.format(ava))
            self.images_ava_hide[ava] = Image.from_file(path + '/images/avatar/{}_h.png'.format(ava))

    def display(self):
        multi = MultiCanvas(2, width=self.client.server['mapsize_x'] * CELL_PIXELS, height=self.client.server['mapsize_y'] * CELL_PIXELS)
        #multi[0].fill_style = 'black'
        #multi[0].fill_rect(0, 0, multi.size[0], multi.size[1])
        self.canvas_base = multi[0]
        self.canvas = multi[1]
        self.output = widgets.Output()
        #display(VBox([Image.from_file(path + '/images/header.jpg', width=200,height=40), HBox([multi])]), self.output)        
        display(multi, self.output)        

    def draw_base(self):
        self.update_map()
        with hold_canvas(self.canvas_base):
            self.canvas_base.clear()
            self.canvas_base.fill_style = 'black'
            self.canvas_base.fill_rect(0, 0, self.canvas_base.size[0], self.canvas_base.size[1])
            for row in range(len(self.map)):
                for col in range(len(self.map[row])):
                    cell = self.map[row][col]
                    if cell['obj'] == obj.wall:
                        self.canvas_base.draw_image(self.images_base[obj.wall], col * CELL_PIXELS, row * CELL_PIXELS)
    
    def redraw(self):
        self.update_map()
        with hold_canvas(self.canvas):
            self.canvas.clear()
            for row in range(len(self.map)):
                for col in range(len(self.map[row])):
                    cell = self.map[row][col]
                    if cell['obj'] == obj.player:
                        self.canvas.draw_image(self.images_ava[cell['image']], col * CELL_PIXELS, row * CELL_PIXELS)
                    elif cell['obj'] == obj.chest:
                        self.canvas.draw_image(self.images_base[obj.chest], col * CELL_PIXELS, row * CELL_PIXELS)

    def update_map(self):
        new_map = map.get_all(self.client.server)
        if self.map is None or self.map != new_map:
            self.map = new_map
    
    def run(self):
        while(True):
            if not self.client.check_state() or self.stop: 
                return
            self.redraw()
            sleep(REDRAW_LAG)

In [7]:
game = Player('4cubes', avatar.lord) 
game.play()
#game.client.drawer = Drawer(game.client)
#game.client.drawer.display()
#game.client.drawer.draw_base()

Ожидание подключения к серверу 4cubes...
Пользователь Артем успешно подключен к серверу 4cubes...


MultiCanvas(height=400, width=400)

Output()

True

In [20]:
game.move_up()
#game.move_left()
#game.move_down()
#game.move_right()
#for _ in range(3):
#    game.move_left()
#game.pick()

True